In [1]:
!python -m spacy download en_core_web_sm

2022-11-12 00:17:18.673123: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 11.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
from collections import OrderedDict
import numpy as np
import spacy
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('punkt')
nlp = spacy.load('en_core_web_sm')
import string


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# RAKE Implementation
import nltk
import string
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)

text = '''
This place was absolutely horrible I thought it would be good but the place was just overall dirty and smelled musty I recommend to anyone that is thinking about going here don't I thought I found something close to a bug in my tamale but wasn't sure so I just threw it away and didn't want to take the chance worst place I have ever been too
'''

# text = '''
# Compatibility of systems of linear constraints over the set of natural numbers.
# Criteria of compatibility of a system of linear Diophantine equations, strict inequations,
# and nonstrict inequations are considered. Upper bounds for components of a minimal set
# of solutions and algorithms of construction of minimal generating sets of solutions for all
# types of systems are given. These criteria and the corresponding algorithms for
# constructing a minimal supporting set of solutions can be used in solving all the
# considered types of systems and systems of mixed types.
# '''

def RakeExtract(text):
        text_tokens = nltk.word_tokenize(text)
        sub_tokens = []

        for word in text_tokens:
            if word in STOP_WORDS or word in string.punctuation:
                sub_tokens.append("-")
            else:
                sub_tokens.append(word)


        spacetokens = []

        res = ""

        for i in range(0,len(sub_tokens)-1):
            if sub_tokens[i] == "-":
                if res != "":
                    spacetokens.append(res)
                spacetokens.append("-")
                res = ""
            if res == "" and sub_tokens[i] != "-":
                res = sub_tokens[i]

            if sub_tokens[i] != "-" and sub_tokens[i+1] != "-":
                res+= " " + sub_tokens[i+1]

        if sub_tokens[-1] != "-":
            res += sub_tokens[-1]
        spacetokens.append(res)


        wrdcnt = {}
        for i in sub_tokens:
            if i!= "-":
                wrdcnt[i] = wrdcnt.get(i,0)+1


        def getinitinfo(tokens):

            wordset = set()
            worddict = {}
            for i in tokens:
                if i != "-":
                    wordset.add(i)

            cnt = 0
            for i in wordset:
                worddict[i] = cnt
                cnt+=1
            return worddict



        dic = getinitinfo(sub_tokens)
        N = len(dic)
        graph = np.zeros((N,N))

        # ["hello world space", "-", "food"]
        for i in spacetokens:
            if i!= "-":
                spl = i.split()
                for i in range(len(spl)):
                    for j in range(i, len(spl)):
                        graph[dic[spl[i]]][dic[spl[j]]] +=1
                        if i!=j:
                            graph[dic[spl[j]]][dic[spl[i]]] +=1

        results = []

        degree = np.sum(graph, axis=0)
        for i, (k,v) in enumerate(dic.items()):

            results.append((k,degree[v]/wrdcnt[k]))

        results.sort(key=lambda x:x[1])
        results = results[::-1]

        keywords = set()
        for word in spacetokens:
            texspl = word.split()
            score = 0
            for x in results:
                for y in texspl:
                    if x[0] == y:
                        score += x[1] 
            keywords.add((word,score))

        keywords = list(keywords)
        keywords.sort(key=lambda x:x[1])
        keywords = keywords[::-1]
        keywords = keywords[:len(keywords)//3]

        return keywords


In [4]:
print(RakeExtract(text))

[('smelled musty I recommend', 15.5), ('absolutely horrible I thought', 15.5), ('I thought I found', 15.0), ('chance worst place I', 13.833333333333334), ('This place', 4.333333333333334), ('overall dirty', 4.0)]
